## Описание проекта

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Импортируем необходимые библиотеки 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import itertools

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from numpy.random import RandomState
RANDOM = 12345
state = RandomState(RANDOM) 

Загрузим и посмторим данные

In [4]:
zero_frame = pd.read_csv("data_0.csv")
first_frame = pd.read_csv("data_1.csv")
second_frame = pd.read_csv("data_2.csv")

In [5]:
zero_frame.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [6]:
first_frame.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
second_frame.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
print(zero_frame.info())
print(first_frame.info())
print(second_frame.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

Пропусков нет, с типами данных все хорошо

In [9]:
print('Количество дубликатов zero_frame:', zero_frame.duplicated().sum())
print('Количество дубликатов first_frame:', first_frame.duplicated().sum())
print('Количество дубликатов second_frame:', second_frame.duplicated().sum())

Количество дубликатов zero_frame: 0
Количество дубликатов first_frame: 0
Количество дубликатов second_frame: 0


С дубликатами тоже все окей, можно идти дальше

## Обучение и проверка модели

Разобьем наши выборки на обучающие и валидационные, обучим модели, получим предсказания и посмотрим на значение ошибки для каждого региона

In [10]:
def split_model(data):
    features = data.drop(["product","id"], axis=1)
    target = data["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, 
                                                                                  random_state=state)
    scaler = StandardScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)
    predicted = pd.Series(predicted)
    rmse = (mean_squared_error(predicted, target_valid))**(0.5)
    mean = sum(predicted) / len(predicted)
    
    print('RMSE:', rmse)
    print('Средний запас предсказанного сырья:', mean)
    
    return (predicted, target_valid.reset_index(drop=True), rmse)

In [11]:
print('Нулевой регион')
predict_zero, target_valid_zero, rmse_zero = split_model(zero_frame)

Нулевой регион
RMSE: 37.5794217150813
Средний запас предсказанного сырья: 92.59256778438005


In [12]:
print('Первый регион')
predict_first, target_valid_first, rmse_first = split_model(first_frame)

Первый регион
RMSE: 0.8897367737680655
Средний запас предсказанного сырья: 68.76995145799783


In [13]:
print('Второй регион')
predict_second, target_valid_second, rmse_second = split_model(second_frame)

Второй регион
RMSE: 39.958042459521614
Средний запас предсказанного сырья: 95.0875281225233


**Вывод:**  
Рейтинг среднего запаса по регионам:  
1. second_frame (94.96)  
2. zero_frame (92.59)
3. first_frame (68.72)

Рейтинг ошибки по регионам:  
1. first_frame (0.89)  
2. zero_frame (37.57)  
3. second_frame (40.02)  

## Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраним в отдельных переменных.

In [14]:
BUDGET = 10**10
MAX_POINTS = 500
POINTS = 200
INCOME = 450000
LOSS_PROBABILITY =  0.025
PLAN = (BUDGET/(INCOME*(10**3)))/POINTS
print('Объем сырья для безубыточной разработки новой скважины:', round(PLAN,3))

Объем сырья для безубыточной разработки новой скважины: 0.111


**Вывод:** 
Объем сырья для безубыточной разработки новой скважины равен 111.111, что выше чем средний запас предсказанного сырья в каждом из регионов.

## Расчёт прибыли и рисков 

In [15]:
def earnings(target_valid, predicted):
    sort = predicted.sort_values(ascending=False)[:POINTS]
    target_sort = target_valid[sort.index][:POINTS]
    target = target_sort.sum() 
    return round(target * INCOME - BUDGET,2)

In [16]:
print('Прибыль в нулевом регионе:', earnings(target_valid_zero,
                                                               predict_zero))
print('Прибыль в первом регионе:', earnings(target_valid_first,
                                                               predict_first))
print('Прибыль во втором регионе:', earnings(target_valid_second,
                                                               predict_second))

Прибыль в нулевом регионе: 3320826043.14
Прибыль в первом регионе: 2415086696.68
Прибыль во втором регионе: 2539915945.84


In [17]:
def confidence(target_valid, predicted):
    samples = []
    for i in range(1000):
        target_sample = target_valid.sample(n=MAX_POINTS, replace=True, random_state=state)
        predictions_sample = predicted[target_sample.index]
        samples.append(earnings(target_valid, predictions_sample))
    samples = pd.Series(samples)
    print('Средняя выручка:', samples.mean())
    print('Риск убытков:', samples.apply(lambda x: x < 0).sum()/len(samples)*100,"%")
    
    lower = samples.quantile(0.025)
    upper = samples.quantile(0.975)
    return round(lower,2), round(upper,2)

In [18]:
print('Доверительный итервал для нулевового региона :',
      confidence(target_valid_zero, predict_zero))
print('Доверительный итервал для первого региона :',
      confidence(target_valid_first, predict_first))
print('Доверительный итервал для второго региона :',
      confidence(target_valid_second, predict_second))

Средняя выручка: 394284411.3404697
Риск убытков: 6.1 %
Доверительный итервал для нулевового региона : (-69450566.85, 915496165.83)
Средняя выручка: 454736359.0974699
Риск убытков: 0.7000000000000001 %
Доверительный итервал для первого региона : (61071811.3, 855914149.14)
Средняя выручка: 353664097.52485
Риск убытков: 7.6 %
Доверительный итервал для второго региона : (-162650946.94, 847761844.51)


## Вывод  

Итак, по результатам исследования для разработки месторождения нам подходит только первый регион, так как риск убытков у него минимальный (и единственный проходящий порог [если я верно посчитала эти риски] ).  
Результаты лучшего региона:  
Средняя выручка: 454736359.09747005  
Риск убытков: 0.7000000000000001 %  
Доверительный итервал для первого региона : (61071811.3, 855914149.14)  